## ResNet50 

### Bottleneck residual block with shorcut reduction

![LeNet Architecture](./images/reduceblock.png)

In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Activation, Add

def bottleneck_residual_block_with_shorcut_reduction(x, kernel_size, filters, strides=(1,1), reduce=False):

    f1, f2, f3 = filters

    if reduce == True:
        shorcut = Conv2D(filters=f3, kernel_size=(1,1), strides=strides, padding='valid')(x)
        shorcut = BatchNormalization()(shorcut)

        main_path = Conv2D(filters=f1, kernel_size=(1,1), strides=strides, padding='valid')(x)
        main_path = BatchNormalization()(main_path)
        main_path = Activation('relu')(main_path)
        
    else:
        shorcut = x
        
        main_path = Conv2D(filters=f1, kernel_size=(1,1), strides=(1,1), padding='valid')(x)
        main_path = BatchNormalization()(main_path)
        main_path = Activation('relu')(main_path)
    
    main_path = Conv2D(filters=f2, kernel_size=kernel_size, strides=(1,1), padding='same')(main_path)
    main_path = BatchNormalization()(main_path)
    main_path = Activation('relu')(main_path)

    main_path = Conv2D(filters=f3, kernel_size=(1,1), strides=(1,1), padding='valid')(main_path)
    main_path = BatchNormalization()(main_path)

    x = Add()([shorcut, main_path])
    x = Activation('relu')(x)

    return x

2024-05-31 12:43:33.309384: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-31 12:43:33.311640: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-31 12:43:33.372290: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-31 12:43:33.585275: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-31 12:43:34.693732: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

### ResNet50 architecture

![LeNet Architecture](./images/resnet50.png)

In [2]:
from tensorflow.keras.layers import Input, AveragePooling2D, Flatten, Dense

input_shape = (32,32,3)
input_layer = Input(shape=input_shape)

#conv1
x = Conv2D(filters=64, kernel_size=(7,7), strides=(2,2), padding='same')(input_layer)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid')(x)

#conv2_x
x = bottleneck_residual_block_with_shorcut_reduction(x, (3,3), [64, 64, 256], strides=(1,1), reduce=True)
x = bottleneck_residual_block_with_shorcut_reduction(x, (3,3), [64, 64, 256])
x = bottleneck_residual_block_with_shorcut_reduction(x, (3,3), [64, 64, 256])

#conv3_x
x = bottleneck_residual_block_with_shorcut_reduction(x, (3,3), [128, 128, 512], strides=(2,2), reduce=True)
x = bottleneck_residual_block_with_shorcut_reduction(x, (3,3), [128, 128, 512])
x = bottleneck_residual_block_with_shorcut_reduction(x, (3,3), [128, 128, 512])
x = bottleneck_residual_block_with_shorcut_reduction(x, (3,3), [128, 128, 512])

#conv4_x
x = bottleneck_residual_block_with_shorcut_reduction(x, (3,3), [256, 256, 1024], strides=(2,2), reduce=True)
x = bottleneck_residual_block_with_shorcut_reduction(x, (3,3), [256, 256, 1024])
x = bottleneck_residual_block_with_shorcut_reduction(x, (3,3), [256, 256, 1024])
x = bottleneck_residual_block_with_shorcut_reduction(x, (3,3), [256, 256, 1024])
x = bottleneck_residual_block_with_shorcut_reduction(x, (3,3), [256, 256, 1024])
x = bottleneck_residual_block_with_shorcut_reduction(x, (3,3), [256, 256, 1024])

#conv5_x
x = bottleneck_residual_block_with_shorcut_reduction(x, (3,3), [512, 512, 2048], strides=(2,2), reduce=True)
x = bottleneck_residual_block_with_shorcut_reduction(x, (3,3), [512, 512, 2048])
x = bottleneck_residual_block_with_shorcut_reduction(x, (3,3), [512, 512, 2048])

x = AveragePooling2D(pool_size=(1,1), strides=(1,1), padding='valid')(x)
x = Flatten()(x)
#x = Dense(1000, activation='relu')(x)
x = Dense(10, activation='softmax')(x) 
resnet50 = Model(inputs=input_layer, outputs=x)
resnet50.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 32, 32, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 16, 16,    │      9,472 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 16, 16,    │        256 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 16, 16,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 7, 7, 64)  │          0 │ activation[0][0]  │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 7, 7, 64)  │      4,160 │ max_pooling2d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 7, 7, 64)  │        256 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 7, 7, 64)  │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 7, 7, 64)  │     36,928 │ activation_1[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 7, 7, 64)  │        256 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 7, 7, 64)  │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 7, 7, 256) │     16,640 │ max_pooling2d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 7, 7, 256) │     16,640 │ activation_2[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 7, 7, 256) │      1,024 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 7, 7, 256) │      1,024 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 7, 7, 256) │          0 │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 7, 7, 256) │          0 │ add[0][0]         │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 7, 7, 64)  │     16,448 │ activation_3[0][… │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 23,608,202 (90.06 MB)

 Trainable params: 23,555,082 (89.86 MB)

 Non-trainable params: 53,120 (207.50 KB)

### Training

In [10]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import SGD

opt = SGD(learning_rate=10e-2, momentum=0.9)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=0.5e-6)
resnet50.compile(loss='categorical_crossentropy', optimizer=opt ,metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='best_weights/resnet50.model.weights.best.keras', save_best_only=True, verbose=1)

#hist = resnet50.fit(x_train, y_train, epoch=200, batch_size=32, validation_data=(x_valid, y_valid), callbacks=[checkpointer, reduce_lr],
#                shuffle=True, verbose=2) 